# XGBoost 순정 모델 테스트

In [1]:
import pandas as pd

## 데이터 로드

In [2]:
data = pd.read_csv("../../data/merged_data.csv", index_col=0)

In [3]:
data.head()

,Rank,Runtime,Weekly Views,n_score,g_score,ky_chart_on,KoreanMovies,Dance,Indie,Trot,...,Ballad,Soul,Rock,Pop,season_autumn,season_spring,season_summer,season_winter,Previous_Rank,New_Rank
0,1,-0.733963,2.119452,0.942971,1.00,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0.000,False
1,2,-0.077307,2.119452,0.942971,1.00,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0.010,False
2,4,-1.390619,2.047278,0.101924,0.06,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0.015,False
3,5,-0.628898,0.083618,0.613535,0.45,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0.025,False
4,6,-1.495684,2.047278,0.101924,0.06,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0.020,False


## 학습 데이터, 평가 데이터 분리

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(data.drop(["Rank"], axis=1), data["Rank"], test_size=0.2, random_state=10)

In [4]:
X_train.head()

,Runtime,Weekly Views,n_score,g_score,ky_chart_on,KoreanMovies,Dance,Indie,Trot,J-POP,...,Ballad,Soul,Rock,Pop,season_autumn,season_spring,season_summer,season_winter,Previous_Rank,New_Rank
9459,0.526816,1.417565,0.275712,0.16,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0.000,False
2681,0.448018,0.343585,0.483764,0.14,0,0,0,1,0,0,...,0,0,0,0,0,0,0,1,0.160,False
475,-0.182372,-0.146799,0.258334,0.00,0,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0.490,False
4709,1.524933,-0.503437,0.590947,0.93,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0.530,False
9057,-0.891560,0.722082,0.254714,0.17,0,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0.675,False


In [5]:
y_train.head()

9459      3
2681     33
475      99
4709    107
9057    188
Name: Rank, dtype: int64

#

In [6]:
from xgboost import XGBRegressor
xgb = XGBRegressor()
eval_set = [(X_train, y_train), (X_test, y_test)]
xgb.fit(X_train, y_train, eval_set=eval_set, verbose=True)

[0]	validation_0-rmse:42.97117	validation_1-rmse:42.97899
[1]	validation_0-rmse:33.04021	validation_1-rmse:33.40147
[2]	validation_0-rmse:26.72375	validation_1-rmse:27.37846
[3]	validation_0-rmse:22.89075	validation_1-rmse:23.90685
[4]	validation_0-rmse:20.52169	validation_1-rmse:21.73466
[5]	validation_0-rmse:19.15243	validation_1-rmse:20.65211
[6]	validation_0-rmse:18.35572	validation_1-rmse:19.96948
[7]	validation_0-rmse:17.80447	validation_1-rmse:19.56881
[8]	validation_0-rmse:17.33259	validation_1-rmse:19.27920
[9]	validation_0-rmse:17.10710	validation_1-rmse:19.10780
[10]	validation_0-rmse:16.78816	validation_1-rmse:18.87382
[11]	validation_0-rmse:16.60097	validation_1-rmse:18.75882
[12]	validation_0-rmse:16.46588	validation_1-rmse:18.73267
[13]	validation_0-rmse:16.17569	validation_1-rmse:18.55786
[14]	validation_0-rmse:15.77248	validation_1-rmse:18.58754
[15]	validation_0-rmse:15.72246	validation_1-rmse:18.58570
[16]	validation_0-rmse:15.70293	validation_1-rmse:18.59743
[17]	va

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [7]:
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# 예측
y_train_pred = xgb.predict(X_train)
y_test_pred = xgb.predict(X_test)

# 평가 지표 계산
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))  # RMSE
train_r2 = r2_score(y_train, y_train_pred)  # R²
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))  # RMSE
test_r2 = r2_score(y_test, y_test_pred)  # R²

print("=== 학습 데이터 평가 지표 ===")
print(f"RMSE: {train_rmse:.2f}")
print(f"R²: {train_r2:.2f}")
print("=== 테스트 데이터 평가 지표 ===")
print(f"RMSE: {test_rmse:.2f}")
print(f"R²: {test_r2:.2f}")

=== 학습 데이터 평가 지표 ===
RMSE: 8.31
R²: 0.98
=== 테스트 데이터 평가 지표 ===
RMSE: 18.72
R²: 0.89
